In [2]:
import midi
import numpy as np

Test python-midi on vectorizing songs.

In [3]:
lowerBound = 24
upperBound = 102
span = upperBound-lowerBound

In [4]:
midifile = '../Jazz_Music_Midi/PianoMan.mid'

In [5]:
pattern = midi.read_midifile(midifile)

In [6]:
pattern

midi.Pattern(format=1, resolution=192, tracks=\
[[midi.SmpteOffsetEvent(tick=0, data=[96, 0, 3, 0, 0]),
  midi.TimeSignatureEvent(tick=0, data=[3, 2, 24, 8]),
  midi.KeySignatureEvent(tick=0, data=[0, 0]),
  midi.SetTempoEvent(tick=0, data=[6, 58, 99]),
  midi.EndOfTrackEvent(tick=0, data=[])],
 [midi.PortEvent(tick=0, data=[0]),
  midi.ProgramChangeEvent(tick=0, channel=0, data=[1]),
  midi.ControlChangeEvent(tick=0, channel=0, data=[7, 127]),
  midi.ControlChangeEvent(tick=0, channel=0, data=[10, 64]),
  midi.NoteOnEvent(tick=0, channel=0, data=[64, 75]),
  midi.NoteOnEvent(tick=0, channel=0, data=[67, 75]),
  midi.NoteOnEvent(tick=192, channel=0, data=[60, 75]),
  midi.NoteOnEvent(tick=160, channel=0, data=[67, 0]),
  midi.NoteOnEvent(tick=0, channel=0, data=[64, 0]),
  midi.NoteOnEvent(tick=32, channel=0, data=[64, 75]),
  midi.NoteOnEvent(tick=0, channel=0, data=[67, 75]),
  midi.NoteOnEvent(tick=160, channel=0, data=[67, 0]),
  midi.NoteOnEvent(tick=0, channel=0, data=[64, 0]),
 

In [7]:
pattern.resolution

192

In [8]:
def midiToStatematrix(midifile, span=span):
    pattern = midi.read_midifile(midifile)

    timeleft = [0 for track in pattern]
    posns = [0 for track in pattern]

    statematrix = []
    time = 0

    state = [[0,0] for x in range(span)]
    statematrix.append(state)

    while True:
        if time % (pattern.resolution/4) == pattern.resolution/8:
            # Crossed a note boundary. Create a new state, defaulting to holding notes
            oldstate = state
            state = [[oldstate[x][0],0] for x in range(span)]
            statematrix.append(state)
        for i in xrange(len(timeleft)): #For each track
            while timeleft[i] == 0:
                track = pattern[i]
                pos = posns[i]

                evt = track[pos]
                if isinstance(evt, midi.NoteEvent):
                    if (evt.pitch < lowerBound) or (evt.pitch >= upperBound):
                        # Ignore note outside range
                        pass
                    else:
                        if isinstance(evt, midi.NoteOffEvent) or evt.velocity == 0:
                            state[evt.pitch-lowerBound] = [0, 0]
                        else:
                            state[evt.pitch-lowerBound] = [1, 1]
                try:
                    timeleft[i] = track[pos + 1].tick
                    posns[i] += 1
                except IndexError:
                    timeleft[i] = None

            if timeleft[i] is not None:
                timeleft[i] -= 1
        
        if all(t is None for t in timeleft):
            break

        time += 1

    mat = np.array(statematrix)
    statematrix = np.hstack((mat[:, :, 0], mat[:, :, 1]))
    statematrix = np.asarray(statematrix).tolist()
    return statematrix

In [9]:
mat = midiToStatematrix(midifile)

In [11]:
mat

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,

Convert state matrix back to MIDI file

In [13]:
def statematrixToMidi(statematrix, name='example', bpm=120):
    pattern = midi.Pattern()
    track = midi.Track()
    pattern.append(track)
    
    tempo = midi.SetTempoEvent(tick=0)
    tempo.set_bpm(bpm)
    track.append(tempo)
    
    tickscale = 55
    
    lastcmdtime = 0
    prevstate = [0 for i in range(span * 2)]
    for time, state in enumerate(statematrix):  
        offNotes = []
        onNotes = []
        for i in range(span):
            if prevstate[i] == 1:
                if state[i] == 0:
                    offNotes.append(i)
                elif state[i + span] == 1:
                    offNotes.append(i)
                    onNotes.append(i)
            elif state[i] == 1:
                onNotes.append(i)
        for note in offNotes:
            track.append(midi.NoteOffEvent(tick=(time-lastcmdtime)*tickscale, pitch=note+lowerBound))
            lastcmdtime = time
        for note in onNotes:
            track.append(midi.NoteOnEvent(tick=(time-lastcmdtime)*tickscale, velocity=40, pitch=note+lowerBound))
            lastcmdtime = time
            
        prevstate = state
    
    eot = midi.EndOfTrackEvent(tick=1)
    track.append(eot)

    midi.write_midifile("{}.mid".format(name), pattern)

In [14]:
statematrixToMidi(mat, bpm=141)

In [15]:
from music21 import converter

In [17]:
new = converter.parse('example.mid')

In [18]:
new.show('midi')

It doesn't sound exact, mostly because we didn't keep some attributes like velocity and cut down the song resolution. It is fairly close though.